In [1]:
import pandas as pd
from nlpia.data.loaders import read_csv

df = read_csv('../shared-resources/chatbot/testset.csv')
df.head()

ValueError: Unable to configure handler 'logging.handlers.SysLogHandler': [Errno 2] No such file or directory